In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, fbeta_score
import timeit

In [34]:
df=pd.read_csv('data10.csv')

In [35]:
df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,category
0,119646801,17,16,597,768,293,0,35.120,97.06,384,...,20,45580.5,32287.848,131880,32165,9922286.0,281712.56,10014561,9030908,0
1,375,2,1,38,0,38,0,19.000,26.88,0,...,20,0.0,0.000,0,0,0.0,0.00,0,0,0
2,281,3,0,46,0,46,0,15.336,26.56,0,...,20,0.0,0.000,0,0,0.0,0.00,0,0,0
3,99999555,2,2,16,0,8,8,8.000,0.00,0,...,32,1.0,0.000,1,1,99999550.0,0.00,99999553,99999553,4
4,10070086,4,4,379,408,379,0,94.750,189.50,408,...,20,0.0,0.000,0,0,0.0,0.00,0,0,0


In [36]:
df.shape

(1033931, 69)

In [37]:
df2=pd.read_csv('2018_best_dtypes.csv')

In [38]:
df2.shape

(69, 2)

In [39]:
df2.head()

,Column,Dtype
0,Flow Duration,int32
1,Tot Fwd Pkts,int32
2,Tot Bwd Pkts,int32
3,TotLen Fwd Pkts,int32
4,TotLen Bwd Pkts,int32


In [40]:
df2.tail()

,Column,Dtype
64,Idle Mean,float32
65,Idle Std,float32
66,Idle Max,int32
67,Idle Min,int32
68,Label,object


In [41]:
df2.loc[68, 'Column']='category'

In [42]:
df2.tail()

,Column,Dtype
64,Idle Mean,float32
65,Idle Std,float32
66,Idle Max,int32
67,Idle Min,int32
68,category,object


In [43]:
mapping = dict(zip(df2['Column'], df2['Dtype']))

In [44]:
# converting into recommended data types

for Column, Dtype in mapping.items():
    df[Column] = df[Column].astype(Dtype)

PART 1: Read data in a pandas dataframe and use dtype of features from file "2018_best_dtypes.csv" file. 
Display in system memory taken by pandas dataframe object.

In [45]:
# displaying memory usage

memory_usage = df.memory_usage(deep=True).sum() / (1024 ** 2) 
print(f"Memory usage of the DataFrame: {memory_usage:.2f} MB")

Memory usage of the DataFrame: 245.52 MB


PART 2: Display value_counts of category column of the dataframe.

In [46]:
df['category'].value_counts()

category
0    901347
3     77595
4     19657
1     14453
5     11382
2      9410
6        87
Name: count, dtype: int64

PART 3:Separate X and y.

In [47]:
X=df.drop(columns=['category'])
y=df['category']

In [48]:
X.shape

(1033931, 68)

In [49]:
y.shape

(1033931,)

PART 4 : Divide data into training (X_train, y_train) and testing (X_test, y_test) data (70% training size), use stratify option.

In [50]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(723751, 68)
(310180, 68)
(723751,)
(310180,)


PART 5: Display value_counts of y_train.

In [51]:
y_train.value_counts()

category
0    630942
3     54317
4     13760
1     10117
5      7967
2      6587
6        61
Name: count, dtype: int64

PART 6: Apply MinMax Scaling (fit on X_train and tranform X_train and X_test).

In [52]:
# MinMax scaling

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

PART 7: Apply PCA (use PCA class from sklearn.decomposition) and keep components till 95% information. [X_train_pca, X_test_pca]

In [53]:
# Principal Component Analysis

pca = PCA(n_components=0.95, random_state=42)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [54]:
X_train_pca.shape

(723751, 9)

In [55]:
X_test_pca.shape

(310180, 9)

In [56]:
X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca=pd.DataFrame(X_test_pca)

In [57]:
X_train_pca.head()

,0,1,2,3,4,5,6,7,8
0,1.020121,-0.307112,-0.650691,-0.206053,0.932307,-0.120064,0.774539,-0.074668,-0.112237
1,0.004703,1.022126,1.032203,-0.232800,0.390737,-0.092835,-0.239461,0.301943,-0.095000
2,-0.948177,1.269459,0.141806,-0.640309,0.237993,0.096651,-0.239707,0.268852,-0.326140
3,-1.244572,-0.122498,-0.348173,0.216850,0.130092,0.385527,-0.054735,-0.069522,-0.024281
4,0.856866,-0.254371,-0.409851,-0.041482,0.287181,-0.158293,-0.433363,-0.007756,0.161872


PART 8: Print variance % of 1st and 2nd principle components.

In [58]:
# finding variance

print(pca.explained_variance_ratio_)

[0.38561526 0.20829335 0.11630425 0.07219939 0.06079518 0.0462549
 0.03275466 0.02329973 0.01637854]


In [59]:
# Decision Tree Classifier

model=DecisionTreeClassifier()

In [60]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [61]:
training_time = timeit.timeit(lambda: model.fit(X_train_pca, y_train), number=1)

In [62]:
# calculating time

start_time = timeit.default_timer()

y_pred = model.predict(X_test_pca)

testing_time = timeit.default_timer() - start_time

PART 9: Apply decision tree classifier. Use timeit library and display time taken by training and testing phase.

In [63]:
# displaying training and testing time

print("Training Time:", training_time, "seconds")
print("Testing Time:", testing_time, "seconds")

Training Time: 25.655939099844545 seconds
Testing Time: 0.07606119988486171 seconds


PART 10: Display Confusion matrix and calculate model accuracy, precision, recall and F-2 score.

In [64]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate accuracy, precision, recall, and F-2 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f2_score = fbeta_score(y_test, y_pred, beta=2, average='weighted')

print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F-2 Score:", f2_score)

Confusion Matrix:
 [[267469     16      0     37     21   2852     10]
 [    16   4320      0      0      0      0      0]
 [     1      0   2820      0      2      0      0]
 [    14      0      0  23263      0      1      0]
 [    19      0      0      0   5878      0      0]
 [  3049      0      0      2      0    363      1]
 [     6      2      0      0      0      0     18]]
Accuracy: 0.9804984202721001
Precision: 0.9799394000881839
Recall: 0.9804984202721001
F-2 Score: 0.9803857546729605
